In [1]:
if (!require("lubridate")) install.packages("lubridate")
if (!require("dplyr")) install.packages("dplyr")
if (!require("ggplot2")) install.packages("ggplot2")

Loading required package: lubridate

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:lubridate’:

    intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


In [2]:
getwd()

[1] "/Users/inder/Dropbox/My Mac (Inders-MacBook-Pro.local)/Documents/GitHub/marketing_datamart/gambeling_analysis"

In [3]:
data = load('DataGroupAssignment.Rdata')

In [17]:
data

[1] "Demographics"         "UserDailyAggregation" "PokerChipConversions"

# data cleaning 

In [4]:
head(Demographics)

UserID,Country,Language,RegDate,FirstPay,FirstAct,FirstSp,FirstCa,FirstGa,FirstPo,ApplicationID,Gender
1324354,276,2,2005-02-01,20050224,20050224,20050224,NULL,NULL,NULL,3,1
1324355,300,8,2005-02-01,20050201,20050201,20050201,NULL,NULL,20050611,1,1
1324356,276,2,2005-02-01,20050201,20050202,20050202,NULL,NULL,NULL,3,1
1324358,752,1,2005-02-01,20050201,20050201,20050201,NULL,NULL,NULL,1,1
1324360,792,7,2005-02-01,20050202,20050202,20050202,20050203,NULL,NULL,9,1
1324362,276,2,2005-02-01,20050211,20050211,20050211,NULL,NULL,NULL,3,1


In [5]:
#check tibble feature types
str(Demographics)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	42649 obs. of  12 variables:
 $ UserID       : num  1324354 1324355 1324356 1324358 1324360 ...
  ..- attr(*, "label")= chr "User ID"
 $ Country      : num  276 300 276 752 792 276 208 276 620 792 ...
  ..- attr(*, "label")= chr "Country of residence"
 $ Language     : num  2 8 2 1 7 2 11 2 16 7 ...
  ..- attr(*, "label")= chr "Primary language"
 $ RegDate      : chr  "2005-02-01" "2005-02-01" "2005-02-01" "2005-02-01" ...
  ..- attr(*, "label")= chr "Registration Date"
 $ FirstPay     : chr  "20050224" "20050201" "20050201" "20050201" ...
  ..- attr(*, "label")= chr "First Payin Date"
 $ FirstAct     : chr  "20050224" "20050201" "20050202" "20050201" ...
  ..- attr(*, "label")= chr "First Active Play Date"
 $ FirstSp      : chr  "20050224" "20050201" "20050202" "20050201" ...
  ..- attr(*, "label")= chr "First Sports Book Play Date"
 $ FirstCa      : chr  "NULL" "NULL" "NULL" "NULL" ...
  ..- attr(*, "label")= chr "First Casino Play Date"
 $ F

In [6]:
#convert registration date to date time 
Demographics$RegDate <- as.Date(Demographics$RegDate)

In [7]:
str(Demographics$RegDate)

 Date[1:42649], format: "2005-02-01" "2005-02-01" "2005-02-01" "2005-02-01" "2005-02-01" ...


In [8]:
#convert first pay, first act, first sports book play, first casino play,
# first games play, and  first poker play dates to datetime.
Demographics$FirstPay <- as.Date(Demographics$FirstPay, "%Y%m%d")

In [9]:
Demographics$FirstAct <- as.Date(Demographics$FirstAct, "%Y%m%d")
Demographics$FirstSp <- as.Date(Demographics$FirstSp, "%Y%m%d")
Demographics$FirstCa <- as.Date(Demographics$FirstCa, "%Y%m%d")
Demographics$FirstGa <- as.Date(Demographics$FirstGa, "%Y%m%d")
Demographics$FirstPo <- as.Date(Demographics$FirstPo, "%Y%m%d")

In [10]:
str(Demographics)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	42649 obs. of  12 variables:
 $ UserID       : num  1324354 1324355 1324356 1324358 1324360 ...
  ..- attr(*, "label")= chr "User ID"
 $ Country      : num  276 300 276 752 792 276 208 276 620 792 ...
  ..- attr(*, "label")= chr "Country of residence"
 $ Language     : num  2 8 2 1 7 2 11 2 16 7 ...
  ..- attr(*, "label")= chr "Primary language"
 $ RegDate      : Date, format: "2005-02-01" "2005-02-01" ...
 $ FirstPay     : Date, format: "2005-02-24" "2005-02-01" ...
 $ FirstAct     : Date, format: "2005-02-24" "2005-02-01" ...
 $ FirstSp      : Date, format: "2005-02-24" "2005-02-01" ...
 $ FirstCa      : Date, format: NA NA ...
 $ FirstGa      : Date, format: NA NA ...
 $ FirstPo      : Date, format: NA "2005-06-11" ...
 $ ApplicationID: num  3 1 3 1 9 3 1 3 1 9 ...
  ..- attr(*, "label")= chr "Application ID"
 $ Gender       : num  1 1 1 1 1 1 1 1 1 1 ...


In [11]:
#create a cutoff date object
october <- as.Date("20051002","%Y%m%d")

In [12]:
#calculate LOR (Length of Relation) based on time since 
#last transaction date in the dataset and the registration date of the 
#gambler
Demographics$LOR <- october - Demographics$RegDate

In [13]:
#calculate play to act interval (PTAI)
#based on time from FirstPay (first betting deposit date)
#and first active date
Demographics$PTAI <- Demographics$FirstAct - Demographics$FirstPay

In [14]:
#calculate registeration to first pay interval (RTFP)
#based on time from gambler registration date to first bet deposit
Demographics$RTFP <- Demographics$FirstPay - Demographics$RegDate

In [15]:
#calculate dummy variable for each first play category (sports book, casino play,
#first games play, first poker play) which indicates whether the indiviual
#has ever partook in these play categories
Demographics$SpPlayed <- ifelse(is.na(Demographics$FirstSp),1,0)

Demographics$CaPlayed <- ifelse(is.na(Demographics$FirstCa),1,0)

Demographics$GaPlayed <- ifelse(is.na(Demographics$FirstGa),1,0)
Demographics$PoPlayed <- ifelse(is.na(Demographics$FirstPo),1,0)

In [16]:
head(Demographics)

UserID,Country,Language,RegDate,FirstPay,FirstAct,FirstSp,FirstCa,FirstGa,FirstPo,ApplicationID,Gender,LOR,PTAI,RTFP,SpPlayed,CaPlayed,GaPlayed,PoPlayed
1324354,276,2,2005-02-01,2005-02-24,2005-02-24,2005-02-24,NA,NA,NA,3,1,243 days,0 days,23 days,0,1,1,1
1324355,300,8,2005-02-01,2005-02-01,2005-02-01,2005-02-01,NA,NA,2005-06-11,1,1,243 days,0 days,0 days,0,1,1,0
1324356,276,2,2005-02-01,2005-02-01,2005-02-02,2005-02-02,NA,NA,NA,3,1,243 days,1 days,0 days,0,1,1,1
1324358,752,1,2005-02-01,2005-02-01,2005-02-01,2005-02-01,NA,NA,NA,1,1,243 days,0 days,0 days,0,1,1,1
1324360,792,7,2005-02-01,2005-02-02,2005-02-02,2005-02-02,2005-02-03,NA,NA,9,1,243 days,0 days,1 days,0,0,1,1
1324362,276,2,2005-02-01,2005-02-11,2005-02-11,2005-02-11,NA,NA,NA,3,1,243 days,0 days,10 days,0,1,1,1


In [20]:
UserDailyAggregation$Date <- as.Date(UserDailyAggregation$Date,"%Y%m%d")

In [23]:
sum(is.na(UserDailyAggregation))

[1] 0

In [24]:
head(UserDailyAggregation)

UserID,Date,ProductID,Stakes,Winnings,Bets
1324354,2005-02-24,1,20,0,2
1324354,2005-02-25,1,0,0,0
1324354,2005-02-27,1,20,0,2
1324354,2005-03-03,1,10,0,1
1324354,2005-03-04,1,10,0,1
1324354,2005-03-05,1,10,0,1


In [76]:
UserDailyAggregation_1 <- UserDailyAggregation %>% 
        group_by(UserID) %>% 
        summarise(first_txn_dt=min(Date),
                  last_txn_dt=max(Date), 
                  txn_cnt=n(), 
                  total_stakes=round(sum(Stakes)), 
                  total_wins=round(sum(Winnings)), 
                  total_bets=round(sum(Bets)), 
                  avg_stakes=round(mean(Stakes)), 
                  avg_wins=round(mean(Winnings)), 
                  avg_bets=round(mean(Bets)))

In [ ]:
head(UserDailyAggregation_1)

UserID,first_txn_dt,last_txn_dt,txn_cnt,total_stakes,total_wins,total_bets,avg_stakes,avg_wins,avg_bets
1324354,2005-02-24,2005-09-30,136,11977,11737,279,88,86,2
1324355,2005-02-01,2005-09-29,106,426,464,252,4,4,2
1324356,2005-02-02,2005-09-12,75,1365,911,214,18,12,3
1324358,2005-02-01,2005-05-06,9,336,210,11,37,23,1
1324360,2005-02-02,2005-09-25,32,66,43,47,2,1,1
1324362,2005-02-11,2005-09-17,7,22,0,7,3,0,1


In [87]:
p_id <- unique(UserDailyAggregation[c("ProductID")])
p_id[order(p_id$ProductID),]

ProductID
1
2
4
5
6
7
8


In [92]:
# Making a new dataframe with number of transactions per UserID in the given period
UserDailyAggregation_2 <- UserDailyAggregation %>% 
                group_by(UserID) %>% 
                summarise(procuct_1_cnt=length(ProductID[ProductID == 1]),
                product_2_cnt=length(ProductID[ProductID == 2]),
                product_4_cnt=length(ProductID[ProductID == 4]),
                product_5_cnt=length(ProductID[ProductID == 5]),
                product_6_cnt=length(ProductID[ProductID == 6]),
                product_7_cnt=length(ProductID[ProductID == 7]),
                product_8_cnt=length(ProductID[ProductID == 8])
                )

In [93]:
head(UserDailyAggregation_2)

UserID,procuct_1_cnt,product_2_cnt,product_4_cnt,product_5_cnt,product_6_cnt,product_7_cnt,product_8_cnt
1324354,117,19,0,0,0,0,0
1324355,99,7,0,0,0,0,0
1324356,51,24,0,0,0,0,0
1324358,8,1,0,0,0,0,0
1324360,29,2,0,0,0,0,1
1324362,7,0,0,0,0,0,0


In [30]:
sapply(UserDailyAggregation_1,function(x) sum(is.na(x)))

UserID first_txn_dt  last_txn_dt      txn_cnt total_stakes   total_wins 
           0            0            0            0            0            0 
  total_bets   avg_stakes     avg_wins     avg_bets 
           0            0            0            0

In [32]:
head(UserDailyAggregation)

UserID,Date,ProductID,Stakes,Winnings,Bets
1324354,2005-02-24,1,20,0,2
1324354,2005-02-25,1,0,0,0
1324354,2005-02-27,1,20,0,2
1324354,2005-03-03,1,10,0,1
1324354,2005-03-04,1,10,0,1
1324354,2005-03-05,1,10,0,1


In [95]:
uda_cleaned <- merge(x = UserDailyAggregation_1, y = UserDailyAggregation_2, by = "UserID", all.x = TRUE)

In [97]:
head(uda_cleaned)

UserID,first_txn_dt,last_txn_dt,txn_cnt,total_stakes,total_wins,total_bets,avg_stakes,avg_wins,avg_bets,procuct_1_cnt,product_2_cnt,product_4_cnt,product_5_cnt,product_6_cnt,product_7_cnt,product_8_cnt
1324354,2005-02-24,2005-09-30,136,11977,11737,279,88,86,2,117,19,0,0,0,0,0
1324355,2005-02-01,2005-09-29,106,426,464,252,4,4,2,99,7,0,0,0,0,0
1324356,2005-02-02,2005-09-12,75,1365,911,214,18,12,3,51,24,0,0,0,0,0
1324358,2005-02-01,2005-05-06,9,336,210,11,37,23,1,8,1,0,0,0,0,0
1324360,2005-02-02,2005-09-25,32,66,43,47,2,1,1,29,2,0,0,0,0,1
1324362,2005-02-11,2005-09-17,7,22,0,7,3,0,1,7,0,0,0,0,0,0


In [99]:
names(uda_cleaned)

[1] "UserID"        "first_txn_dt"  "last_txn_dt"   "txn_cnt"      
 [5] "total_stakes"  "total_wins"    "total_bets"    "avg_stakes"   
 [9] "avg_wins"      "avg_bets"      "procuct_1_cnt" "product_2_cnt"
[13] "product_4_cnt" "product_5_cnt" "product_6_cnt" "product_7_cnt"
[17] "product_8_cnt"